In [15]:
import fitsio as fio
import numpy as np

In [16]:
mem = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog_members.fit')[-1] # Member catalog
meta = fio.FITS('/lsst/troxel/y1a1/y1a1-im3shape_v5_unblind_v2_matched_v4.fits')[-1] # Shape catalog

In [17]:
# Read galaxy ID columns
mem_id = mem['ID'].read()
meta_id = meta['coadd_objects_id'].read()

In [18]:
def sort2n(x,y):
    """
    Sorts and matches two arrays of object ids where x is unique and y is not (in DES this is coadd_objects_id).
    Slower than sort2().
    """
    
    xsort = np.argsort(x)
    ysort = np.argsort(y)
    i_yx = np.sort(y[np.in1d(y, x, assume_unique=False)])
    i_x = xsort[x[xsort].searchsorted(i_yx)]
    i_y = ysort[y[ysort].searchsorted(i_yx)]
    
    return i_x, i_y

In [19]:
# Sort and mask member catalog
mem_sort = np.argsort(mem_id)
mem_id = mem_id[mem_sort]
mem_mask = np.in1d(mem_id, meta_id, assume_unique=False)
mem_id = mem_id[mem_mask]

In [20]:
# Sort shape catalog
ix, iy = sort2n(meta_id, mem_id)
meta_id = meta_id[ix]

In [21]:
mem_dtypes = mem.read().dtype.descr # Member catalog column names

In [22]:
# Create new catalog with useful columns
new_array = np.zeros(len(mem_id), dtype=mem_dtypes + [('e1',float)] + [('e2',float)] + [('flags_select',int)])

In [23]:
# Fill catalog with member catalog and useful shape data
for name in mem.read().dtype.names:
    new_array[name] = mem.read()[name][mem_sort][mem_mask]
for name in ['e1','e2','flags_select']:
    new_array[name] = meta[name].read()[ix]

In [24]:
fio.write('/lsst/troxel/y1a1/lgt20_member_im3shape_combined.fits', new_array, clobber=True)

In [25]:
flags_mask = new_array['flags_select']==0
shape_masked = new_array[flags_mask]

In [26]:
fio.write('/lsst/troxel/y1a1/lgt20_member_im3shape_combined_masked.fits', shape_masked, clobber=True)